In [2]:
%%javascript
if(MathJax) {
    MathJax.Hub.Config({
        TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
    MathJax.Hub.Queue(
    ["resetEquationNumbers", MathJax.InputJax.TeX],
    ["PreProcess", MathJax.Hub],
    ["Reprocess", MathJax.Hub]
  );
}

<IPython.core.display.Javascript object>

## ニューラルネットワークの学習

[データ] -> (人の考えたアルゴリズム) -> [答え]
[データ] -> (人の考えた特徴量) -> (機械学習 SVM, KNNなど） -> [答え]
[データ] -> (ニューラルネットワーク（ディープラーニング) -> [答え]

ニューラルネットワーク（ディープラーニング）は，特徴量の抽出という人間の介在がなくとも，入力からそのまま学習することができる．

### 損失関数 loss function or cost function

1. Mean Squared Error 二乗和誤差

NNの出力と教師ラベルの誤差を二乗して合計したもの

$$
\begin{equation}
E = \frac{1}{2} \sum_k(y_k-t_k)^2
\end{equation}
$$

2. Cross Entropy Error 交差エントロピー誤差

$$
\begin{equation}
E = -\sum_k t_k log y_k
\end{equation}
$$

$t_k$は，正解ラベルとなるインデックスだけが1で，その他は0であるというone-hot表現であるとする．
とすると，この式はつまり，正解した出力のlogを合計したものである．
$log 1 = 0$であることから，すべての正解ラベルに対して，$y_k=1$を出力できた場合，つまり全問正解のとき，$E=0$となる．
これに対して，正解ラベルに対して，$y_k<1$を出力したものが誤差として加算されていく．



In [5]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t) ** 2)

# deltaを足すのは，log(0) = -infになるのを避けるため
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

## ミニバッチ学習

訓練データすべてから損失関数の和をもとめるには，訓練データのサンプル数をNとして，交差エントロピー誤差の場合，上記式を拡張して，

$$
\begin{equation}
E = - \frac{1}{N} \sum_n \sum_k t_{nk} log y_{nk}
\end{equation}
$$

となる．ｍnistの場合，訓練データが約60000あり，個々のデータにはk個の出力があるわけなので，なかなか時間がかかってしまう．
そこで，訓練データの中からサンプリングすることで近似値を求めながら学習させるのが，ミニバッチ学習である．

## エポック

エポックとは，訓練データを一通りすべて使いきった回数．10000個の訓練データをミニバッチサイズ1000で訓練させていたら，ミニバッチ10回で，1エポックとなる．

## なぜ誤差関数を設定するのか？

mnistの場合は，正解かどうかの「認識精度」をそのまま指標とすればよいのではないだろうか？
A. 学習時に最適なパラメータを探索する際には，微分が用いられるが，認識精度を指標にしてしまうと，ほとんどのところで０になり，微分ができず，うまく学習（パラメータ更新）ができないからである．
正答率を指標にした場合，その指標の変化は非連続である（離散である）．しかし，損失関数を用いると，あたかも精度が連続値のように扱うことができ，微分可能になるのである．

## 勾配の求め方

通常学習するパラメータは複数あるため，それぞれのパラメータの偏微分を求めてから，それらをベクトルとしてまとめたものを，勾配という

In [7]:
import numpy as np

def function_2(x):
    return np.sum(x**2)

def numerical_gradient(f, x):
    h = 1e-4 # 微分に用いるdelta
    grad = np.zeros_like(x)
    for idx in range(x.size):
        # 各パラメータごとに偏微分を求める
        tmp = x[idx]
        
        x[idx] = tmp + h
        fxh1 = f(x)

        x[idx] = tmp - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp
    return grad

print(numerical_gradient(function_2, np.array([3.0, 4.0])))

[6. 8.]


## 勾配法

最適なパラメータとは，損失関数が最小となる場合であり，勾配の示す方向が，その損失を最小化する方向である．

勾配法は，この勾配の進む方向にある一定距離勧め，移動した先でまた勾配を評価して進む，ということを繰り返して，最小値にたどり着く手法である．

Gradient descent method = 勾配降下法

$$
\begin{equation}
x_0 = x_0 - \eta \frac{\partial f}{\partial x_0} \\
x_1 = x_1 - \eta \frac{\partial f}{\partial x_1}
\end{equation}
$$

$\eta$ は，learning rate = 学習率．ハイパーパラメータとして事前に設定する必要がある．



In [8]:
# 勾配法の実装
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

In [10]:
# x_0^2 + x_1^2 の最小値を求める
def func(x):
    return np.sum(x**2)

init_x = np.array([3.0, 4.0])
gradient_descent(func, init_x, lr=0.1) # 結果，だいたい(0, 0)に近い値が求められる

array([6.11110793e-10, 8.14814391e-10])

## ニューラルネットワークの勾配

ある層において，

$$
\begin{equation}
W = \begin{pmatrix}
  w_{11} & w_{12} & w_{13} \\
  w_{21} & w_{22} & w_{23}
\end{pmatrix}
\end{equation}
$$

というWeightが与えられ，その損失が，$L$であるとすると，

$$
\begin{equation}
\frac{ \partial L }{ \partial W } = \begin{pmatrix}
  \frac{ \partial L }{ \partial w_{11}} & \frac{ \partial L }{ \partial w_{12} } & \frac{ \partial L }{ \partial w_{13} } \\
  \frac{ \partial L }{ \partial w_{21}} & \frac{ \partial L }{ \partial w_{22} } & \frac{ \partial L }{ \partial w_{23} }
\end{pmatrix}
\end{equation}
$$

が，その損失の勾配となる．



In [13]:
import sys, os
import os.path as path
sys.path.append(path.abspath(path.join(os.curdir ,"../..")))

from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

## Simpleなニューラルネットの実装
class SimpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        return cross_entropy_error(y, t)

net = SimpleNet()
print(net.W)


[[-0.85962715  0.26073598 -0.36334443]
 [-1.07225197 -1.06688374 -1.02781945]]


In [16]:
x = np.array([0.6, 0.9])
t = np.array([0, 1, 0])
print(net.predict(x))
print(net.loss(x, t))

[-1.48074306 -0.80375378 -1.14304417]
0.7976962138947823


In [18]:
def f(W):
    return net.loss(x, t)

# 勾配を求める
dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.137304   -0.32977709  0.19247309]
 [ 0.205956   -0.49466563  0.28870963]]


## ニューラルネットの学習の実装

0. (前提)ニューラルネットには，「重み」と「バイアス」というパラメータがある
1. 損失関数
2. ミニバッチ
3. 勾配
4. 勾配降下法（ミニバッチを使うため，確率的勾配降下法=stochastic gradient descent sgdという）

を組み合わせることで，実装可能．まずは，2層のネットワークからやってみよう

In [23]:
from common.functions import sigmoid, softmax, cross_entropy_error
from common.gradient import numerical_gradient

class Tom2Net:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        a1 = np.dot(x, self.params['W1']) + self.params['b1']
        z1 = sigmoid(a1)
        a2 = np.dot(z1, self.params['W2']) + self.params['b2']
        return softmax(a2)
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accu = np.sum(y == t) / float(x.shape[0])
        return accu
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        pass

## Sample usage
n = Tom2Net(28**2, 50, 10)
x = np.random.rand(100, 28**2) ## dummy data x 100
t = np.random.rand(100, 10) ## dummy label * 100
y = n.predict(x)
grads = n.numerical_gradient(x, t)
print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

## Train in mini-batch
from dataset.mnist import load_mnist
(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

## Hyper params
iter_num = 10_000
train_size = x_train.shape[0]
batch_size = 100 # ミニバッチの数
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)

net = Tom2Net(28**2, 50, 10)

for i in range(iter_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]

    grad = net.numerical_gradient(x_batch, y_batch)
    for key in ['W1', 'b1', 'W2', 'b2']:
        n.params[key] -= learning_rate * grad[key]
    
    # Training 履歴の記録
    loss = net.loss(x_batch, y_batch)
    train_loss_list.append(loss)

    # 1epochごとにaccuracyを更新
    if i % iter_per_epoch == 0:
        train_acc = net.accuracy(x_train, y_train)
        test_acc = net.accuracy(x_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"train acc, test acc | {str(train_acc)}, {str(test_acc)}")



(784, 50)
(50,)
(50, 10)
(10,)


TypeError: load_mnist() got an unexpected keyword argument 'one_host_label'

しかし，この`numerical_gradient`はあまりに遅すぎる．．というわけで，次章では，勾配を求める偏微分を高速化する「誤差逆伝播法」（Back propagation）について学ぶ